In [7]:
import numpy as np
import pandas as pd

# Draft Elips

In [ ]:
#    t1 t2 .. tn
# l1 xx xx    xx r1
# l2 xx xx    xx r2
# ..
# lm xx xx    xx rm
#    b1 b2    bn

# row => m
# col => n

In [12]:
m = 1
n = 2

# kalo nilainya sama pake ini
t = np.full(n, 5)
r = np.full(m, 55)
b = np.full(n, 25)
l = np.full(m, 30)

In [14]:
matrix = np.zeros((m+2, n+2))
matrix

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

# Parabola

In [94]:
# misal kasus
# 150 ? ? ? 75

# env = suhu lingkungan (suhu awal)
# l   = suhu kiri
# r   = suhu kanan
# I   = ? ada ...
# J   = sampai j ke-...
# q   = lambda

## Metode Eksplisit

Menghitung masa depan dari masa lalu

In [87]:
def IterateExplicitParabol(Tj, j, I, J, q):    
    Tjj = Tj.copy()
    
    for i in range(1, I+1):
        Tjj[i] = Tj[i] + q*(Tj[i-1] - 2*Tj[i] + Tj[i+1])
        
    if j < J:
        return IterateExplicitParabol(Tjj, j+1, I, J, q)
    
    return Tjj
    

def ExplicitParabol(env, l, r, I, J, q):
    Tj0 = np.concatenate((l, np.full(I, env), r), axis=None, dtype="float32")
    
    return IterateExplicitParabol(Tj0, 1, I, J, q)


ExplicitParabol(env=25, l=150, r=75, I=3, J=2, q=0.0835)

array([150.      ,  44.13194 ,  26.220144,  32.652775,  75.      ],
      dtype=float32)

## Metode Implisit

In [99]:
def CreateMatrixImplicitParabol(T, q):
    I = T.size
    A = np.zeros((I, I))
    B = np.zeros(I)
    
    for i in range(1, I-1):
        B[i] += T[i]
        
        if i == 1:
            B[i] += q*T[i-1]
        else:
            A[i, i-1] = -q
            
        A[i, i] = 1 + 2*q
            
        if i+2 == I:
            B[i] += q*T[i+1]
        else:
            A[i, i+1] = -q
            
    return [A[1:I-1, 1:I-1], B[1:I-1]]


def IterateImplicitParabol(Tj, j, J, q):
    A, B = CreateMatrixImplicitParabol(Tj, q)
    X = np.linalg.solve(A, B)
    
    Tjj = np.concatenate((Tj[0], X, Tj[-1]), axis=None, dtype="float32")
    
    if j < J:
        return IterateImplicitParabol(Tjj, j+1, J, q)
    
    return Tjj    
    

def ImplicitParabol(env, l, r, I, J, q):
    Tj0 = np.concatenate((l, np.full(I, env), r), axis=None, dtype="float32")    
    
    return IterateImplicitParabol(Tj0, 1, J, q)


ImplicitParabol(env=25, l=150, r=75, I=3, J=2, q=0.0835)

array([150.      ,  41.84027 ,  27.472544,  31.875557,  75.      ],
      dtype=float32)

## Metode Crank-Nicolson

In [98]:
def CreateMatrixCrankNicolson(T, q):
    I = T.size
    A = np.zeros((I, I))
    B = np.zeros(I)
    
    for i in range(1, I-1):
        B[i] += q*T[i-1] + 2*(1-q)*T[i] + q*T[i+1]
        
        if i == 1:
            B[i] += q*T[i-1]
        else:
            A[i, i-1] = -q
            
        A[i, i] = 2 + 2*q
            
        if i+2 == I:
            B[i] += q*T[i+1]
        else:
            A[i, i+1] = -q
            
    return [A[1:I-1, 1:I-1], B[1:I-1]]


def IterateCrankNicolson(Tj, j, J, q):
    A, B = CreateMatrixCrankNicolson(Tj, q)
    X = np.linalg.solve(A, B)
    
    Tjj = np.concatenate((Tj[0], X, Tj[-1]), axis=None, dtype="float32")
    
    if j < J:
        return IterateCrankNicolson(Tjj, j+1, J, q)
    
    return Tjj    


def CrankNicolson(env, l, r, I, J, q):
    Tj0 = np.concatenate((l, np.full(I, env), r), axis=None, dtype="float32")    
    
    return IterateCrankNicolson(Tj0, 1, J, q)


CrankNicolson(env=25, l=150, r=75, I=3, J=2, q=0.0835)

array([150.     ,  42.89295,  26.92991,  32.22404,  75.     ],
      dtype=float32)